In [1]:
import sys

sys.path.append("../../")

from astronn import datasets
from astronn import models

import matplotlib.pyplot as plt

from matplotlib.pyplot import figure
from scipy.signal import find_peaks, peak_widths


seed_value = 12345
import os

os.environ["PYTHONHASHSEED"] = str(seed_value)
import random
import numpy as np

random.seed(seed_value)
np.random.seed(seed_value)
import tensorflow as tf
tf.autograph.set_verbosity(0)

# Star model to train
star_models = datasets.starmodels()

df_train = star_models.load(
    #"/home/roberto/Downloads/evolutionTracks_line/parts_train/*_norm",
    "/home/roberto/Downloads/evolutionTracks_line/norm8/*_norm",
    batch_size=500,
    add_noise=True,
)
df_validation = star_models.load(
    #"/home/roberto/Downloads/evolutionTracks_line/parts_validation/*_norm",
    "/home/roberto/Downloads/evolutionTracks_line/nomr8_validation/*_norm",
    batch_size=150,
    add_noise=True,
)


# Read datasets of preprocessed real stars to test
df_ds = datasets.deltascuti()
deltascuti = df_ds.load(
    "../../astronn/data/deltascuti/preprocessed/*", batch_size=1
)
bedding_stars = datasets.bedding()
df_bedding = bedding_stars.load("../data/bedding/preprocessed/*", batch_size=1)

# Get stars
ds_stars = [star for star in deltascuti.take(11)]
ds_bedding = [star for star in df_bedding.take(57)]

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set

In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [3]:
"""

# Learn and autoencoder from star models
from tensorflow.keras import layers
from keras.models import load_model

encoding_dim = 2

autoencoder = tf.keras.Sequential(
    [
        layers.Input(shape=(400, 2)),
        layers.Flatten(),
        layers.Dense(200),
        layers.Dense(
            encoding_dim,
            name="lattent",
            activity_regularizer=tf.keras.regularizers.l1(10e-5),
        ),
        layers.Dense(200),
        layers.Dense(800),
        layers.Reshape((400, 2)),
    ]
)

# This model maps an input to its reconstruction
opt = tf.keras.optimizers.Adam(learning_rate=0.1)
autoencoder.compile(loss="binary_crossentropy", optimizer=opt, metrics=["mse"])
autoencoder.summary()

autoencoder.fit(
    df_train, steps_per_epoch=40, epochs=10, verbose=True,
)
"""

'\n\n# Learn and autoencoder from star models\nfrom tensorflow.keras import layers\nfrom keras.models import load_model\n\nencoding_dim = 2\n\nautoencoder = tf.keras.Sequential(\n    [\n        layers.Input(shape=(400, 2)),\n        layers.Flatten(),\n        layers.Dense(200),\n        layers.Dense(\n            encoding_dim,\n            name="lattent",\n            activity_regularizer=tf.keras.regularizers.l1(10e-5),\n        ),\n        layers.Dense(200),\n        layers.Dense(800),\n        layers.Reshape((400, 2)),\n    ]\n)\n\n# This model maps an input to its reconstruction\nopt = tf.keras.optimizers.Adam(learning_rate=0.1)\nautoencoder.compile(loss="binary_crossentropy", optimizer=opt, metrics=["mse"])\nautoencoder.summary()\n\nautoencoder.fit(\n    df_train, steps_per_epoch=40, epochs=10, verbose=True,\n)\n'

In [4]:
"""
count = 0
for star in df_train.take(1):
    plt.plot(star[0][0, :, 0])
    plt.plot(star[0][0, :, 1])
    plt.show()
    
plt.plot(autoencoder.predict(star[0])[0])

autoencoder.get_layer("lattent").output
autoencoder.save("/tmp/autoencoder")
"""

'\ncount = 0\nfor star in df_train.take(1):\n    plt.plot(star[0][0, :, 0])\n    plt.plot(star[0][0, :, 1])\n    plt.show()\n    \nplt.plot(autoencoder.predict(star[0])[0])\n\nautoencoder.get_layer("lattent").output\nautoencoder.save("/tmp/autoencoder")\n'

In [5]:
#model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#    filepath="/tmp/model_checkpoint",
#    save_weights_only=True,
#    monitor='val_acc',
#    mode='max',
#    save_best_only=True)

In [6]:
def get_peak_width(position, peaks, peaks_width, peaks_sorted_by_prob):
    """
    """
    peak_id = np.where(probs == peaks_sorted_by_prob[position])[0][0]
    return (peak_id, peaks_width[0][np.where(peaks == peak_id)[0][0]])

In [7]:
results = {}
results["loss"] = []
results["accuracy"] = []
results["val_loss"] = []
results["val_accuracy"] = []

results["binaries_errors"] = []
results["binaries_errors_top2"] = []
results["binaries_mse"] = []

results["bedding_errors"] = []
results["bedding_errors_top2"] = []
results["bedding_mse"] = []


_index = np.array([i for i in range(100)], dtype=np.int32)
_index
class_weight_dict = {}
for e in _index:
    class_weight_dict[int(e ) * 1.0] = 0.2
print(class_weight_dict)


higher_n_probs = 10
loops = 1
for loop in range(loops):
    print("Loop %s" % loop)
    sepconv_mod = models.separableconvnn()  # init model
    sepconv_mod.compile(learning_rate=0.01)  # compile model

    for ep in range(300 + 1):
        print("Episode %s" % ep)
        history = sepconv_mod.model.fit(
            # df_train,
            df_train,
            validation_data=df_validation,
            validation_steps=1,
            steps_per_epoch=10,
            epochs=1,
            #class_weight=class_weight_dict,
            verbose=True,
        )

        if ep % 20 == 0:
            sepconv_mod.save("/tmp/sepconvnn_tmp_dos_%s" % ep)  # tmp model save

        results["loss"].append(history.history["loss"][0])
        results["accuracy"].append(history.history["accuracy"][0])
        results["val_loss"].append(history.history["val_loss"][0])
        results["val_accuracy"].append(history.history["val_accuracy"][0])

{0.0: 0.2, 1.0: 0.2, 2.0: 0.2, 3.0: 0.2, 4.0: 0.2, 5.0: 0.2, 6.0: 0.2, 7.0: 0.2, 8.0: 0.2, 9.0: 0.2, 10.0: 0.2, 11.0: 0.2, 12.0: 0.2, 13.0: 0.2, 14.0: 0.2, 15.0: 0.2, 16.0: 0.2, 17.0: 0.2, 18.0: 0.2, 19.0: 0.2, 20.0: 0.2, 21.0: 0.2, 22.0: 0.2, 23.0: 0.2, 24.0: 0.2, 25.0: 0.2, 26.0: 0.2, 27.0: 0.2, 28.0: 0.2, 29.0: 0.2, 30.0: 0.2, 31.0: 0.2, 32.0: 0.2, 33.0: 0.2, 34.0: 0.2, 35.0: 0.2, 36.0: 0.2, 37.0: 0.2, 38.0: 0.2, 39.0: 0.2, 40.0: 0.2, 41.0: 0.2, 42.0: 0.2, 43.0: 0.2, 44.0: 0.2, 45.0: 0.2, 46.0: 0.2, 47.0: 0.2, 48.0: 0.2, 49.0: 0.2, 50.0: 0.2, 51.0: 0.2, 52.0: 0.2, 53.0: 0.2, 54.0: 0.2, 55.0: 0.2, 56.0: 0.2, 57.0: 0.2, 58.0: 0.2, 59.0: 0.2, 60.0: 0.2, 61.0: 0.2, 62.0: 0.2, 63.0: 0.2, 64.0: 0.2, 65.0: 0.2, 66.0: 0.2, 67.0: 0.2, 68.0: 0.2, 69.0: 0.2, 70.0: 0.2, 71.0: 0.2, 72.0: 0.2, 73.0: 0.2, 74.0: 0.2, 75.0: 0.2, 76.0: 0.2, 77.0: 0.2, 78.0: 0.2, 79.0: 0.2, 80.0: 0.2, 81.0: 0.2, 82.0: 0.2, 83.0: 0.2, 84.0: 0.2, 85.0: 0.2, 86.0: 0.2, 87.0: 0.2, 88.0: 0.2, 89.0: 0.2, 90.0: 0.2, 91.0: 0.

Episode 20
10/10 [==============================] - 4s 372ms/step - loss: 2.6889 - accuracy: 0.2490 - val_loss: 10.0106 - val_accuracy: 0.0000e+00
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
INFO:tensorflow:Assets written to: /tmp/sepconvnn_tmp_dos_20/assets
Episode 21
10/10 [==============================] - 4s 383ms/step - loss: 2.6967 - accuracy: 0.2456 - val_loss: 8.3088 - val_accuracy: 0.0000e+00
Episode 22
10/10 [=======

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
INFO:tensorflow:Assets written to: /tmp/sepconvnn_tmp_dos_80/assets
Episode 81
10/10 [==============================] - 4s 373ms/step - loss: 2.4514 - accuracy: 0.2850 - val_loss: 2.3425 - val_accuracy: 0.3467
Episode 82
10/10 [==============================] - 4s 373ms/step - loss: 2.4355 - accuracy: 0.2966 - val_loss: 3.0555 - val_accuracy: 0.2400
Episode 83
10/10 [==============================] - 4s 374ms/step - loss: 2.4583 - accuracy: 0.2940 - val_loss: 2.2920 - val_accuracy: 0.3000
Episode 84
10/10 [==============================] - 4s 369ms/step - loss: 2.4859 - accuracy: 0.2750 - val_loss: 2.4856 - val_accuracy: 0.2600
Episode 85
10/10 [==============================] - 4

10/10 [==============================] - 4s 368ms/step - loss: 2.2956 - accuracy: 0.3134 - val_loss: 2.0433 - val_accuracy: 0.4000
Episode 122
10/10 [==============================] - 4s 370ms/step - loss: 2.3140 - accuracy: 0.3106 - val_loss: 2.1920 - val_accuracy: 0.3067
Episode 123
10/10 [==============================] - 4s 368ms/step - loss: 2.3144 - accuracy: 0.3206 - val_loss: 1.9860 - val_accuracy: 0.3533
Episode 124
10/10 [==============================] - 4s 372ms/step - loss: 2.2729 - accuracy: 0.3056 - val_loss: 2.2586 - val_accuracy: 0.3067
Episode 125
10/10 [==============================] - 4s 370ms/step - loss: 2.2760 - accuracy: 0.3082 - val_loss: 2.4462 - val_accuracy: 0.3133
Episode 126
10/10 [==============================] - 4s 372ms/step - loss: 2.2500 - accuracy: 0.3152 - val_loss: 2.3191 - val_accuracy: 0.2933
Episode 127
10/10 [==============================] - 5s 455ms/step - loss: 2.2722 - accuracy: 0.3172 - val_loss: 1.9898 - val_accuracy: 0.3333
Episode 128

Episode 165
10/10 [==============================] - 5s 449ms/step - loss: 2.1858 - accuracy: 0.3326 - val_loss: 1.9956 - val_accuracy: 0.3133
Episode 166
10/10 [==============================] - 4s 374ms/step - loss: 2.2230 - accuracy: 0.3114 - val_loss: 1.8891 - val_accuracy: 0.4067
Episode 167
10/10 [==============================] - 4s 372ms/step - loss: 2.2244 - accuracy: 0.3240 - val_loss: 2.0833 - val_accuracy: 0.3867
Episode 168
10/10 [==============================] - 4s 371ms/step - loss: 2.2147 - accuracy: 0.3310 - val_loss: 1.8856 - val_accuracy: 0.4000
Episode 169
10/10 [==============================] - 4s 371ms/step - loss: 2.2141 - accuracy: 0.3250 - val_loss: 1.9486 - val_accuracy: 0.3667
Episode 170
10/10 [==============================] - 4s 371ms/step - loss: 2.1772 - accuracy: 0.3320 - val_loss: 1.9355 - val_accuracy: 0.3867
Episode 171
10/10 [==============================] - 4s 372ms/step - loss: 2.2089 - accuracy: 0.3352 - val_loss: 1.9263 - val_accuracy: 0.3667

10/10 [==============================] - 4s 386ms/step - loss: 2.1537 - accuracy: 0.3308 - val_loss: 2.1438 - val_accuracy: 0.3000
Episode 209
10/10 [==============================] - 5s 501ms/step - loss: 2.1530 - accuracy: 0.3370 - val_loss: 2.2105 - val_accuracy: 0.3133
Episode 210
10/10 [==============================] - 4s 392ms/step - loss: 2.1720 - accuracy: 0.3300 - val_loss: 2.1127 - val_accuracy: 0.3333
Episode 211
10/10 [==============================] - 4s 401ms/step - loss: 2.1421 - accuracy: 0.3440 - val_loss: 2.0771 - val_accuracy: 0.3533
Episode 212
10/10 [==============================] - 4s 384ms/step - loss: 2.1672 - accuracy: 0.3338 - val_loss: 1.8566 - val_accuracy: 0.4533
Episode 213
10/10 [==============================] - 4s 390ms/step - loss: 2.1527 - accuracy: 0.3326 - val_loss: 1.7511 - val_accuracy: 0.3933
Episode 214
10/10 [==============================] - 4s 387ms/step - loss: 2.1710 - accuracy: 0.3276 - val_loss: 1.8666 - val_accuracy: 0.3600
Episode 215

Episode 252
10/10 [==============================] - 4s 377ms/step - loss: 2.1513 - accuracy: 0.3316 - val_loss: 1.8949 - val_accuracy: 0.3933
Episode 253
10/10 [==============================] - 4s 369ms/step - loss: 2.1424 - accuracy: 0.3320 - val_loss: 1.9743 - val_accuracy: 0.4067
Episode 254
10/10 [==============================] - 4s 374ms/step - loss: 2.1071 - accuracy: 0.3334 - val_loss: 2.1210 - val_accuracy: 0.3467
Episode 255
10/10 [==============================] - 4s 370ms/step - loss: 2.0941 - accuracy: 0.3370 - val_loss: 1.8504 - val_accuracy: 0.4067
Episode 256
10/10 [==============================] - 4s 371ms/step - loss: 2.1049 - accuracy: 0.3386 - val_loss: 2.0576 - val_accuracy: 0.3200
Episode 257
10/10 [==============================] - 4s 372ms/step - loss: 2.1390 - accuracy: 0.3378 - val_loss: 1.9283 - val_accuracy: 0.3867
Episode 258
10/10 [==============================] - 4s 366ms/step - loss: 2.1196 - accuracy: 0.3354 - val_loss: 1.8903 - val_accuracy: 0.3067

10/10 [==============================] - 4s 374ms/step - loss: 2.1163 - accuracy: 0.3430 - val_loss: 1.9538 - val_accuracy: 0.3400
Episode 296
10/10 [==============================] - 4s 371ms/step - loss: 2.0988 - accuracy: 0.3386 - val_loss: 1.8903 - val_accuracy: 0.3667
Episode 297
10/10 [==============================] - 4s 373ms/step - loss: 2.0870 - accuracy: 0.3438 - val_loss: 1.9273 - val_accuracy: 0.3333
Episode 298
10/10 [==============================] - 4s 374ms/step - loss: 2.0821 - accuracy: 0.3462 - val_loss: 1.9020 - val_accuracy: 0.3333
Episode 299
10/10 [==============================] - 4s 371ms/step - loss: 2.0885 - accuracy: 0.3556 - val_loss: 1.8622 - val_accuracy: 0.3600
Episode 300
10/10 [==============================] - 4s 369ms/step - loss: 2.0830 - accuracy: 0.3390 - val_loss: 1.9464 - val_accuracy: 0.3533
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full ou

In [8]:
class_weight_dict

{0.0: 0.2,
 1.0: 0.2,
 2.0: 0.2,
 3.0: 0.2,
 4.0: 0.2,
 5.0: 0.2,
 6.0: 0.2,
 7.0: 0.2,
 8.0: 0.2,
 9.0: 0.2,
 10.0: 0.2,
 11.0: 0.2,
 12.0: 0.2,
 13.0: 0.2,
 14.0: 0.2,
 15.0: 0.2,
 16.0: 0.2,
 17.0: 0.2,
 18.0: 0.2,
 19.0: 0.2,
 20.0: 0.2,
 21.0: 0.2,
 22.0: 0.2,
 23.0: 0.2,
 24.0: 0.2,
 25.0: 0.2,
 26.0: 0.2,
 27.0: 0.2,
 28.0: 0.2,
 29.0: 0.2,
 30.0: 0.2,
 31.0: 0.2,
 32.0: 0.2,
 33.0: 0.2,
 34.0: 0.2,
 35.0: 0.2,
 36.0: 0.2,
 37.0: 0.2,
 38.0: 0.2,
 39.0: 0.2,
 40.0: 0.2,
 41.0: 0.2,
 42.0: 0.2,
 43.0: 0.2,
 44.0: 0.2,
 45.0: 0.2,
 46.0: 0.2,
 47.0: 0.2,
 48.0: 0.2,
 49.0: 0.2,
 50.0: 0.2,
 51.0: 0.2,
 52.0: 0.2,
 53.0: 0.2,
 54.0: 0.2,
 55.0: 0.2,
 56.0: 0.2,
 57.0: 0.2,
 58.0: 0.2,
 59.0: 0.2,
 60.0: 0.2,
 61.0: 0.2,
 62.0: 0.2,
 63.0: 0.2,
 64.0: 0.2,
 65.0: 0.2,
 66.0: 0.2,
 67.0: 0.2,
 68.0: 0.2,
 69.0: 0.2,
 70.0: 0.2,
 71.0: 0.2,
 72.0: 0.2,
 73.0: 0.2,
 74.0: 0.2,
 75.0: 0.2,
 76.0: 0.2,
 77.0: 0.2,
 78.0: 0.2,
 79.0: 0.2,
 80.0: 0.2,
 81.0: 0.2,
 82.0: 0.2,
 83.0: 0.2,
 8

In [9]:
x = []
y = []
l = []
for star in ds_stars:
    target = np.where(star[2].numpy().flat == 1)[0].flat[0]
    l.append(star[0].numpy()[0].decode("utf-8"))
    probs = sepconv_mod.model.predict_proba(star[1])[0]

    print(l)
    plt.plot(probs)
    plt.show()

    plt.plot(np.arange(0, 100, 0.25), star[1][0, :, 0], label="dft", color="blue")
    # plt.plot(x, star[1][0, :, 1], label="hod", color="orange", alpha=0.5)
    plt.plot(np.arange(0, 100, 0.25), star[1][0, :, 1], label="ac", color="green")
    plt.show()
    peaks, _ = find_peaks(probs, height=0, distance=5)
    peaks_width = peak_widths(probs, peaks)
    peaks_sorted_by_prob = np.sort(probs[peaks])[::-1]
    best_peak, best_peak_width = get_peak_width(
        0, peaks, peaks_width, peaks_sorted_by_prob
    )
    x.append(best_peak)
    y.append(target)

plt.scatter(y, x, label=l)
print(np.asarray(x) - np.asarray(y))
print(np.mean(np.asarray(x) - np.asarray(y)))
print(np.mean(np.power(np.asarray(x) - np.asarray(y), 2)))
print(x)
print(y)
print(l)

AttributeError: 'Sequential' object has no attribute 'predict_proba'

In [ ]:
plt.plot(results["loss"])
plt.plot(results["val_loss"])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "validation"], loc="upper left")
plt.show()

plt.plot(results["accuracy"])
plt.plot(results["val_accuracy"])
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(["train", "validation"], loc="upper left")
plt.show()

In [ ]:
x = [i for i in range(len(results["binaries_errors"]))]
y = results["binaries_errors"]
m, b = np.polyfit(x, y, 1)
plt.plot(x, m * np.asarray(x) + b)

In [ ]:
x = [i for i in range(len(results["bedding_errors"]))]
y = results["bedding_errors"]
m, b = np.polyfit(x, y, 1)
plt.plot(x, m * np.asarray(x) + b)

In [ ]:
targets = {}
targets["kic10661783"] = {"dnu": 39.0, "dr": 7.0}
targets["KIC9851944"] = {"dnu": 26.0, "dr": 5.3}
targets["HD159561"] = {"dnu": 38.0, "dr": 19.0}
targets["CID100866999"] = {"dnu": 56, "dr": np.nan}
targets["HD15082"] = {"dnu": 80.0, "dr": 14.0}
targets["kic4544587"] = {"dnu": 74.0, "dr": 11.0}
targets["KIC8262223"] = {"dnu": 77.0, "dr": 7.10}
targets["HD172189"] = {"dnu": 19.0, "dr": 4.6}
targets["KIC3858884"] = {"dnu": 19.0, "dr": 1.9}
targets["CID105906206"] = {"dnu": 20.0, "dr": 2.61}
targets["KIC10080943"] = {"dnu": 52.0, "dr": 1.7}

import matplotlib.cm as cm
import matplotlib.colors as mcolors

# setup the normalization and the colormap
normalize = mcolors.Normalize(vmin=0, vmax=higher_n_probs)
colormap = plt.get_cmap("jet_r")
# setup the colorbar
scalarmappaple = cm.ScalarMappable(norm=normalize, cmap=colormap)
scalarmappaple.set_array(higher_n_probs)

for star in results.keys():
    if star in targets.keys():
        for idx, val in enumerate(range(higher_n_probs)):
            plt.plot(
                results[star][(val,)],
                alpha=0.5,
                marker="o",
                linestyle="None",
                color=colormap(normalize(idx)),
                markersize=higher_n_probs - idx,
            )
        plt.xlabel("NN epoch traininng (class infered)")
        plt.ylabel(r"Frequency ($\mu$hz)")
        plt.axhline(y=targets[star]["dnu"], color="black")
        plt.title(star)
        plt.ylim(0, 100)
        cbar = plt.colorbar(scalarmappaple)
        cbar.set_label("Inference order (by probability)")
        plt.show()